# Working with data

This section runs through the API for common tasks in machine learning.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

PyTorch offers domain-specific libraries such as TorchText, TorchVision, and TorchAudio, all of which include datasets. For this tutorial, we will be using a TorchVision dataset.

The torchvision.datasets module contains Dataset objects for many real-world vision data like CIFAR, COCO (full list here). In this tutorial, we use the FashionMNIST dataset. Every TorchVision Dataset includes two arguments: transform and target_transform to modify the samples and labels respectively.

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|████████████████████████████████████████████████████████████████▊| 26345472/26421880 [00:14<00:00, 1230497.61it/s]

Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



0it [00:00, ?it/s]
  0%|                                                                                        | 0/29515 [00:00<?, ?it/s]
32768it [00:01, 30949.30it/s]                                                                                          


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



0it [00:00, ?it/s]
 97%|████████████████████████████████████████████████████████████████▋  | 4268032/4422102 [00:03<00:00, 1920529.22it/s]

Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw




0it [00:00, ?it/s]

8192it [00:00, 9878.84it/s]                                                                                            


Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw
Processing...
Done!


26427392it [00:30, 1230497.61it/s]                                                                                     
4423680it [00:23, 1920529.22it/s]                                                                                      

We pass the Dataset as an argument to DataLoader. This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


# Creating Models

To define a neural network in PyTorch, we create a class that inherits from nn.Module. We define the layers of the network in the __init__ function and specify how data will pass through the network in the forward function. To accelerate operations in the neural network, we move it to the GPU if available.

In [20]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv = nn.Conv2d(1, 32, 3)
        self.maxpool = nn.MaxPool2d(2)
        self.dropout = nn.Dropout2d(0.2)
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5408, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv(x)
        x = nn.ReLU()(x)
        x = self.maxpool(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        output = self.softmax(x)
        return output

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout2d(p=0.2, inplace=False)
  (flatten): Flatten()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=5408, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=1)
)


# Optimizing the Model Parameters

To train a model, we need a loss function and an optimizer.

In [30]:
loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters())

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

In [31]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model’s performance against the test dataset to ensure it is learning.

In [32]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model’s accuracy and loss at each epoch; we’d like to see the accuracy increase and the loss decrease with every epoch.

In [33]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 1.632518  [    0/60000]
loss: 1.683216  [ 6400/60000]
loss: 1.647257  [12800/60000]
loss: 1.739015  [19200/60000]
loss: 1.614339  [25600/60000]
loss: 1.629323  [32000/60000]
loss: 1.616244  [38400/60000]
loss: 1.635328  [44800/60000]
loss: 1.619895  [51200/60000]
loss: 1.610565  [57600/60000]
Test Error: 
 Accuracy: 85.9%, Avg loss: 0.025171 

Epoch 2
-------------------------------
loss: 1.525426  [    0/60000]
loss: 1.625374  [ 6400/60000]
loss: 1.527720  [12800/60000]
loss: 1.643857  [19200/60000]
loss: 1.570369  [25600/60000]
loss: 1.648443  [32000/60000]
loss: 1.568945  [38400/60000]
loss: 1.613071  [44800/60000]
loss: 1.637200  [51200/60000]
loss: 1.603144  [57600/60000]
Test Error: 
 Accuracy: 87.0%, Avg loss: 0.024988 

Epoch 3
-------------------------------
loss: 1.526050  [    0/60000]
loss: 1.586429  [ 6400/60000]
loss: 1.513326  [12800/60000]
loss: 1.627096  [19200/60000]
loss: 1.576628  [25600/60000]
loss: 1.640435  [32000/600

# Saving Models

A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [35]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


# Loading Models

The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [36]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

This model can now be used to make predictions.

In [45]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]

x = x.unsqueeze(0)

with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
